In [ ]:
'''
NOTE :
Some Parts of this code like training script and dataloader preparation are borrowed from source : https://www.kaggle.com/code/bilalafzal255/toxic-comments-classification-using-transformers

## TED Lens BERT V2

In [ ]:
!pip install transformers -v

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


Importing Libraries

Data Preprocessing

In [ ]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,multilabel_confusion_matrix
import numpy as np
import torch
import seaborn as sns

import gensim
import gensim.downloader as api


In [ ]:
data = pd.read_csv('/content/ted_talks_en.csv')

xtraining_column = 'description'
useful_data = data[[xtraining_column, 'topics']]
useful_data.head()

,description,topics
0,With the same humor and humanity he exuded in ...,"['alternative energy', 'cars', 'climate change..."
1,You've never seen data presented like this. Wi...,"['Africa', 'Asia', 'Google', 'demo', 'economic..."
2,New York Times columnist David Pogue takes aim...,"['computers', 'entertainment', 'interface desi..."
3,"In an emotionally charged talk, MacArthur-winn...","['MacArthur grant', 'activism', 'business', 'c..."
4,Sir Ken Robinson makes an entertaining and pro...,"['children', 'creativity', 'culture', 'dance',..."


In [ ]:
print(data['description'][1],'\n',data['topics'][1])

You've never seen data presented like this. With the drama and urgency of a sportscaster, statistics guru Hans Rosling debunks myths about the so-called "developing world." 
 ['Africa', 'Asia', 'Google', 'demo', 'economics', 'global issues', 'health', 'statistics', 'global development', 'visualizations', 'math']


Calculating the text length and its margin of error.

In [ ]:
import scipy.stats as stats
import math
import statistics

def moe(token_lengths, confidence_level=0.95):
    """
    Calculate the margin of error for the average token length.

    Parameters:
    - token_lengths (list): List of token lengths.
    - confidence_level (float): Desired confidence level (default is 0.95 for a 95% confidence interval).

    Returns:
    - float: Margin of error for the average token length.
    """

    # Sample size
    n = len(token_lengths)

    # Sample standard deviation
    sample_std_dev = statistics.stdev(token_lengths)

    # Calculate t-score for the confidence level (use n-1 degrees of freedom)
    t_score = stats.t.ppf((1 + confidence_level) / 2, df=n - 1)

    # Calculate margin of error for the average token length
    margin_of_error = t_score * (sample_std_dev / math.sqrt(n))

    return margin_of_error


# Example usage:
token_lengths_description = [len(i.split()) for i in data['description']]
token_lengths_transcript = [len(i.split()) for i in data['transcript']]
topics_lengths = [len(eval(i)) for i in data['topics']]
confidence_level = 0.95

print(f"Average_token_length Description : {statistics.mean(token_lengths_description)} +_ {moe(token_lengths_description, confidence_level)}")
print(f"Average_token_length transcript : {statistics.mean(token_lengths_transcript)} +_ {moe(token_lengths_transcript, confidence_level)}")
print(f"Average_length topics : {statistics.mean(topics_lengths)} +_ {moe(topics_lengths, confidence_level)}")


Average_token_length Description : 58.19550561797753 +_ 0.6019476476687287
Average_token_length transcript : 1796.845692883895 +_ 31.10394621545405
Average_length topics : 7.915106117353308 +_ 0.12095621044348982


Exploring the Topics available in the Dataset

In [ ]:
# Load the CSV dataset
data = useful_data

# converting the text to
all_topics = []
for topics,i in zip(data['topics'],range(len(data['topics']))):
    all_topics+=eval(topics)
    data['topics'][i] = eval(topics)

unique_topics = sorted(list(set(all_topics)))
print(unique_topics)

num_labels = len(unique_topics)
print(num_labels,' Topics')

['3D printing', 'AI', 'AIDS', 'Africa', "Alzheimer's", 'Antarctica', 'Anthropocene', 'Asia', 'Audacious Project', 'Autism spectrum disorder', 'Best of the Web', 'Brand', 'Brazil', 'Buddhism', 'CRISPR', 'Christianity', 'DNA', 'Debate', 'Egypt', 'Europe', 'Foreign Policy', 'Gender spectrum', 'God', 'Google', 'HIV', 'Humanities', 'Internet', 'Iran', 'Islam', 'LGBT', 'Latin America', 'MacArthur grant', 'Mars', 'Middle East', 'Moon', 'NASA', 'New York', 'Nobel Prize', 'PTSD', 'Planets', 'Science (hard)', 'Senses', 'Slavery', 'Social Science', 'South America', 'String theory', 'Sun', 'Surgery', 'Syria', 'TED Books', 'TED Connects', 'TED Fellows', 'TED Prize', 'TED Residency', 'TED en Español', 'TED-Ed', 'TEDMED', 'TEDNYC', 'TEDYouth', 'TEDx', 'Transgender', 'United States', 'Vaccines', 'activism', 'addiction', 'adventure', 'advertising', 'aging', 'agriculture', 'aircraft', 'algorithm', 'alternative energy', 'ancient world', 'animals', 'animation', 'anthropology', 'ants', 'apes', 'archaeology

Function to convert a DataFrame column to its BERT Embeddings

In [ ]:
data.head()

,description,topics
0,With the same humor and humanity he exuded in ...,"[alternative energy, cars, climate change, cul..."
1,You've never seen data presented like this. Wi...,"[Africa, Asia, Google, demo, economics, global..."
2,New York Times columnist David Pogue takes aim...,"[computers, entertainment, interface design, m..."
3,"In an emotionally charged talk, MacArthur-winn...","[MacArthur grant, activism, business, cities, ..."
4,Sir Ken Robinson makes an entertaining and pro...,"[children, creativity, culture, dance, educati..."


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
topics_labels = mlb.fit_transform(data['topics'])

for i in range(len(data['topics'])):
    data['topics'][i] = topics_labels[i]

In [ ]:
data.head()

,description,topics
0,With the same humor and humanity he exuded in ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,You've never seen data presented like this. Wi...,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,New York Times columnist David Pogue takes aim...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"In an emotionally charged talk, MacArthur-winn...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Sir Ken Robinson makes an entertaining and pro...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
from torch.utils.data import Dataset, DataLoader

class MultiLabelDataset(Dataset):

    def __init__(self, df, tokenizer, max_len, new_data=False):
        self.tokenizer = tokenizer
        self.data = df
        self.text = df.description
        self.new_data = new_data
        self.max_len = max_len

        if not new_data:
            self.targets = self.data.topics

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])

        inputs = self.tokenizer.encode_plus(text,None,
                                           add_special_tokens=True,
                                           max_length=self.max_len,
                                           pad_to_max_length=True,
                                           return_token_type_ids=True)
        out = {
            "input_ids": torch.tensor(inputs['input_ids'], dtype=torch.long),
            "attention_mask": torch.tensor(inputs['attention_mask'], dtype=torch.long),
            "token_type_ids": torch.tensor(inputs['token_type_ids'], dtype=torch.long)
        }
        if not self.new_data:
            out["targets"] = torch.tensor(self.targets[index], dtype=torch.float)

        return out

## Model Testing and Evaluation

In [ ]:
MAX_LEN = 512
EPOCHS = 100
LEARNING_RATE = 1e-05
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda:0'

In [ ]:
import os
import random
import torch

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

## Preparing Data for the Model

In [ ]:
train_size = 0.7

train_df, df_temp = train_test_split(data, test_size=0.3, random_state=42)

val_df, test_df = train_test_split(df_temp, test_size=0.1, random_state=42)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
train_set = MultiLabelDataset(train_df, tokenizer, MAX_LEN)
val_set = MultiLabelDataset(val_df, tokenizer, MAX_LEN)
test_set = MultiLabelDataset(test_df, tokenizer, MAX_LEN)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=8)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=8)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=8)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Model Architecture

In [ ]:
from transformers import DistilBertModel
from torch import nn

class DistilBertClass(torch.nn.Module):
    def __init__(self):
        super(DistilBertClass, self).__init__()

        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")

        # Convolutional layer
        self.conv1d = nn.Conv1d(in_channels=768, out_channels=766, kernel_size=3, stride=1, padding=1)

        # Activation function
        self.relu = nn.ReLU()

        # Linear layers for classification
        self.classifier = nn.Sequential(
            nn.Linear(766, 457),
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]

        # Applying Conv1d layer without selecting a specific token
        out = self.conv1d(hidden_state.transpose(1, 2))
        out = self.relu(out)

        # Performing global max pooling to reduce sequence dimension
        out = nn.functional.adaptive_max_pool1d(out, 1).squeeze()

        # Apply Linear layers for classification
        out = self.classifier(out)

        return out


In [ ]:
model = DistilBertClass()
model.to(DEVICE)
print(f"Model on {DEVICE}")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Model on cuda:0


In [ ]:
# Load the pretrained weights from a .pth file
model_path = 'saved_model_v2.pth'
checkpoint = torch.load(model_path)

# Load the state_dict into the model
model.load_state_dict(checkpoint)

<All keys matched successfully>

## Display Confusion Function

In [ ]:
import matplotlib.pyplot as plt

def display_confusion(confusion_matrix):
  for matrix,label in zip(confusion_matrix,unique_topics):
    plt.figure(figsize=(4, 4))
    sns.heatmap(matrix, annot=True, cmap='Blues')
    plt.title(label)
    plt.close()


## Testing

In [ ]:
from sklearn import metrics
from tqdm.auto import tqdm

def test(last=False):
  model.eval()

  fin_targets = []
  fin_outputs = []

  with torch.inference_mode():
    for _, data in tqdm(enumerate(test_loader, 0)):
      ids = data['input_ids'].to(DEVICE, dtype=torch.long)
      mask = data['attention_mask'].to(DEVICE, dtype=torch.long)
      token_type_ids = data['token_type_ids'].to(DEVICE, dtype=torch.long)
      targets = data['targets'].to(DEVICE, dtype=torch.float)

      outputs = model(ids, mask, token_type_ids)
      fin_targets.extend(targets.cpu().detach().numpy().tolist())
      fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

  fin_outputs = np.array(fin_outputs) >= 0.25
  fin_targets = np.array(fin_targets) >= 0.5
  accuracy = metrics.accuracy_score(fin_targets, fin_outputs)
  f1_score_micro = metrics.f1_score(fin_targets, fin_outputs, average='micro',zero_division=1.0)
  f1_score_macro = metrics.f1_score(fin_targets, fin_outputs, average='macro',zero_division=1.0)
  precision = metrics.precision_score(fin_targets, fin_outputs, average='macro',zero_division=1.0)
  recall = metrics.recall_score(fin_targets, fin_outputs, average='macro',zero_division=1.0)

  if last:
    confusion_matrix = multilabel_confusion_matrix(fin_targets, fin_outputs)
    display_confusion(confusion_matrix)


  for i in range(len(fin_targets)):
    print(test_df[xtraining_column][i])
    print(sorted(mlb.inverse_transform(fin_targets)[i]))
    print(sorted(mlb.inverse_transform(fin_outputs)[i]))
    print()

  targets_sum = 0
  outputs_sum = 0

  for i,j in zip(mlb.inverse_transform(fin_targets), mlb.inverse_transform(fin_outputs)):
    targets_sum += len(i)
    outputs_sum += len(j)

  print(f"Target Average {targets_sum/len(fin_targets)} Output Average {outputs_sum/len(fin_outputs)}")

  return {
      "Accuracy Score": accuracy,
      "F1 score(micro)": f1_score_micro,
      "F1 score(macro)": f1_score_macro,
      "Precision":precision,
      "Recall":recall
  }

In [ ]:
test()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to m

0it [00:00, ?it/s]

Someone has tripped the magical alarms in the Element Temple. When you and the other monks arrive on the scene, you know you have a disaster on your hands. Four young apprentices broke into the temple's inner chamber to steal the sacred element crystals. But when the alarm went off they panicked, and each of them swallowed a crystal. Can you determine who ate which crystal? Alex Gendler shows how. [Directed by Artrake Studio, narrated by Addison Anderson].
['TED-Ed', 'animation', 'education', 'math']
['TED-Ed', 'animation', 'math', 'science']

Long before we had rocket scientists, the idea of spaceflight traveled from mind to mind across generations. With great visuals, TED Fellow and NASA economist Alexander MacDonald shows how 300 years of sci-fi tales -- from Edgar Allan Poe to Jules Verne to H.G. Wells and beyond -- sparked a culture of space exploration. A fascinating look at how stories become reality, featuring a goose machine sent to the Moon.
['Moon', 'NASA', 'Planets', 'TED F

{'Accuracy Score': 0.0,
 'F1 score(micro)': 0.45488126649076516,
 'F1 score(macro)': 0.5114853467296125,
 'Precision': 0.710083200583695,
 'Recall': 0.6004151577745348}